**Problem3 AC-GAN**

In [ ]:
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets import cifar10
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot as plt
from keras.preprocessing import image
import numpy as np


**Import necessary libraries**

In [ ]:
def plot_diagram(disc_real, disc_fake, gen_loss, epoch):
  plt.figure(figsize=(15, 6))
  plt.plot(list(range(1, epoch + 2)),disc_real, linewidth=3)
  plt.plot(list(range(1, epoch + 2)),disc_fake, linewidth=3)
  plt.plot(list(range(1, epoch + 2)),gen_loss, linewidth=3)
  plt.legend(['disc_real','disc_fake','gen_loss'])

  plt.title("Loss Values")
  plt.ylabel("Loss")
  plt.xticks(list(range(1, epoch + 1)))
  plt.xlabel('epoch')
  plt.grid(which='both', axis='both')
  plt.savefig('/content/drive/My Drive/NeuralNet/Q3/AC_GAN/Plots/plot_Epoch_%d.png' % epoch, bbox_inches='tight')
  plt.close()


**Loss-Function Diagram**

In [ ]:

# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3), n_classes=10):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # image input
  in_image = Input(shape=in_shape)
  fe = Conv2D(16, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  fe = Conv2D(32, (3,3), strides=(1,1), padding='same', kernel_initializer=init)(in_image)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  fe = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  fe = Conv2D(128, (3,3), strides=(1,1), padding='same', kernel_initializer=init)(fe)
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  fe = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  fe = Conv2D(512, (3,3), strides=(1,1), padding='same', kernel_initializer=init)(fe)
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  # flatten feature maps
  fe = Flatten()(fe)
  # real/fake output
  out1 = Dense(1, activation='sigmoid')(fe)
  # class label output
  out2 = Dense(n_classes, activation='softmax')(fe)
  # define model
  model = Model(in_image, [out1, out2])
  # compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
  return model

**Define Discriminator: Someone can find detail descriotion of this architecture in Mini-Project Report**

In [ ]:
def define_generator(latent_dim, n_classes=10):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # label input
  in_label = Input(shape=(1,))
  # embedding for categorical input
  li = Embedding(n_classes, 50)(in_label)
  # linear multiplication
  n_nodes = 8 * 8 * 3
  li = Dense(n_nodes, kernel_initializer=init)(li)
  # reshape to additional channel
  li = Reshape((8, 8, 3))(li)
  # image generator input
  in_lat = Input(shape=(latent_dim,))
  n_nodes = 384 * 8 * 8
  gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
  gen = Activation('relu')(gen)
  gen = Reshape((8, 8, 384))(gen)
  # merge image gen and label input
  merge = Concatenate()([gen, li])
  gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
  gen = BatchNormalization()(gen)
  gen = Activation('relu')(gen)
  gen = Conv2DTranspose(96, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
  gen = BatchNormalization()(gen)
  gen = Activation('relu')(gen)
  gen = Conv2DTranspose(3, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  out_layer = Activation('tanh')(gen)
  model = Model([in_lat, in_label], out_layer)
  return model

**Define Generator: This part we implement architecture of the paper**

In [ ]:
def define_gan(g_model, d_model):
  # Freez Dicriminator
  d_model.trainable = False
  gan_output = d_model(g_model.output)
  model = Model(g_model.input, gan_output)
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
  return model


**Define AC-GAN Model by merging Generator and Discriminator**

In [ ]:

def load_real_samples():
  # load dataset
  (trainX, trainy), (_, _) = cifar10.load_data()
  X = trainX
  X = X.astype('float32')
  X = (X - 127.5) / 127.5
  print(X.shape, trainy.shape)
  return [X, trainy]

def generate_real_samples(dataset, n_samples):
  images, labels = dataset
  ix = randint(0, images.shape[0], n_samples)
  X, labels = images[ix], labels[ix]
  y = ones((n_samples, 1))
  return [X, labels], y

def generate_latent_points(latent_dim, n_samples, n_classes=10):
  x_input = randn(latent_dim * n_samples)
  z_input = x_input.reshape(n_samples, latent_dim)
  labels = randint(0, n_classes, n_samples)
  return [z_input, labels]

def generate_fake_samples(generator, latent_dim, n_samples):
  z_input, labels_input = generate_latent_points(latent_dim, n_samples)
  images = generator.predict([z_input, labels_input])
  y = zeros((n_samples, 1))
  return [images, labels_input], y
def summarize_performance(step, g_model, latent_dim, n_samples=100):
  [X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
  fig, axs = plt.subplots(10, 10, figsize=(15,15))
  plt.subplots_adjust(hspace=0.0, wspace=0.0)
  NUMBER_PER_CLASS = 10
  for classlabel in range(10):
    gen_imgs = X
    for i in range(NUMBER_PER_CLASS):
         #Dont scale the images back, let keras handle it#
         img = image.array_to_img(gen_imgs[classlabel*10 + i], scale=True)
         axs[i,classlabel].imshow(img)
         axs[i,classlabel].axis('off')
  fig.savefig('/content/drive/My Drive/NeuralNet/Q3/AC_GAN/Epoch_%d.png' % step, bbox_inches='tight')
  plt.close()

**Some useful functions for loading and preprocessing the data and genrate output of Generator and plot diagrams and outputs if Generator part**

In [ ]:

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=500, n_batch=64):
  bat_per_epo = int(dataset[0].shape[0] / n_batch)
  n_steps = bat_per_epo * n_epochs
  # calculate the size of half a batch of samples
  half_batch = int(n_batch / 2)
  disc_real_tmp = 0
  disc_fake_tmp = 0
  _epoch = 0
  gen_tmp = 0
  disc_real = []
  disc_fake = []
  gen_loss = []
  for i in range(n_steps):
    # get randomly selected 'real' samples
    [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
    # update discriminator model weights
    _,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
    # generate 'fake' examples
    [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
    # update discriminator model weights
    _,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
    # prepare points in latent space as input for the generator
    [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
    # create inverted labels for the fake samples
    y_gan = ones((n_batch, 1))
    # update the generator via the discriminator's error
    _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
    disc_real_tmp += d_r2
    disc_fake_tmp += d_f2
    gen_tmp += g_2

    if (i+1)%bat_per_epo == 0:
      disc_real.append(disc_real_tmp/bat_per_epo)
      disc_fake.append(disc_fake_tmp/bat_per_epo)
      gen_loss.append(gen_tmp/bat_per_epo)
      summarize_performance(i, g_model, latent_dim)
      plot_diagram(disc_real, disc_fake, gen_loss, _epoch)
      disc_real_tmp = 0
      disc_fake_tmp = 0
      gen_tmp = 0
      _epoch +=1

# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

(50000, 32, 32, 3) (50000, 1)


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29
Epoch:  30
Epoch:  31
Epoch:  32
Epoch:  33
Epoch:  34
Epoch:  35
Epoch:  36
Epoch:  37
Epoch:  38
Epoch:  39
Epoch:  40
Epoch:  41
Epoch:  42
Epoch:  43
Epoch:  44
Epoch:  45
Epoch:  46
Epoch:  47
Epoch:  48
Epoch:  49
Epoch:  50
Epoch:  51
Epoch:  52
Epoch:  53
Epoch:  54
Epoch:  55
Epoch:  56
Epoch:  57
Epoch:  58
Epoch:  59
Epoch:  60
Epoch:  61
Epoch:  62
Epoch:  63
Epoch:  64
Epoch:  65
Epoch:  66
Epoch:  67
Epoch:  68
Epoch:  69
Epoch:  70
Epoch:  71
Epoch:  72
Epoch:  73
Epoch:  74
Epoch:  75
Epoch:  76
Epoch:  77
Epoch:  78
Epoch:  79
Epoch:  80
Epoch:  81
Epoch:  82
Epoch:  83
Epoch:  84
Epoch:  85
Epoch:  86
Epoch:  87
Epoch:  88
Epoch:  89
Epoch:  90
Epoch:  91
Epoch:  

**Train Model in 500 Epoch. in each batch we must first train Discriminator on real and fake images. Then by freezing Discriminaotr(In gan function we freezed them) train the Generator.**